<a href="https://colab.research.google.com/github/adirsonmfjr/credito-rural/blob/main/Vs01_Credito_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import requests
import pandas as pd
from io import BytesIO

# URL do arquivo no GitHub
url = "https://raw.githubusercontent.com/adirsonmfjr/credito-rural/main/Dados_Estoque_credito_rural_1993-2023.xls"

# Baixar o arquivo
response = requests.get(url)

if response.status_code == 200:
    # Carregar o Excel com ajustes
    df = pd.read_excel(
        BytesIO(response.content),
        sheet_name="Séries",  # Nome da aba no Excel
        engine="xlrd",
        dtype=str  # Lê como string para evitar conversões erradas
    )

    # Converter colunas numéricas para float e substituir NaN por 0
    for col in df.columns[3:]:  # Pular as três primeiras colunas (Sigla, Código, Município)
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    print("Arquivo carregado com sucesso!")

else:
    print("Erro ao baixar o arquivo:", response.status_code)

# Configurar pandas para exibir valores sem notação científica
pd.set_option('display.float_format', '{:,.0f}'.format)

# Exibir as primeiras linhas para verificar se faz sentido
df.head()


Arquivo carregado com sucesso!


,Sigla,Codigo,Município,1993,1994,1995,1996,1997,1998,1999,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AC,1200013,Acrelândia,0,0,21749,5083039,1180168,3603147,3962375,...,20776771,8562961,11362390,11235470,14613649,15877934,9088736,8996591,6580221,4908745
1,AC,1200054,Assis Brasil,125701,1764378,294535,0,0,244387,688827,...,3197656,35660,334018,5179684,1636721,924183,297710,524263,112330,151097
2,AC,1200104,Brasiléia,6372470,5486466,3067611,486008,818832,1006212,2966766,...,8906259,9917391,16527385,13863737,11236307,10363271,7850346,3423072,3098155,3913477
3,AC,1200138,Bujari,0,0,569863,13844,108046,293693,635708,...,16035479,18792652,9158576,10670679,17666324,14845991,6806347,9281610,4009833,3910707
4,AC,1200179,Capixaba,0,0,0,0,104771,560449,0,...,4519761,3421325,4075607,3578121,11337150,6119740,3826153,2452279,6232797,3550908


In [22]:
# Identificar as colunas de anos (ignorar as colunas categóricas)
colunas_anos = [col for col in df.columns if col.isdigit()]  # Filtra apenas colunas com nome numérico

# Criar colunas de Data Inicial e Data Final
df["Data_Inicial"] = int(min(colunas_anos))  # Primeiro ano disponível na base
df["Data_Final"] = int(max(colunas_anos))    # Último ano disponível na base

# Criar a coluna de Recência (R)
def calcular_recencia(row):
    anos_validos = [int(ano) for ano in colunas_anos if row[ano] > 0]  # Filtra os anos com estoque maior que 0

    if anos_validos:
        return df["Data_Final"].iloc[0] - max(anos_validos)  # Último ano de estoque - Data Final
    return df["Data_Final"].iloc[0] - df["Data_Inicial"].iloc[0]  # Caso não tenha estoque, pega o período total

df["Recencia"] = df.apply(calcular_recencia, axis=1)

# Criar a coluna de Frequência (F) - Contagem de anos com estoque positivo
df["Frequencia"] = (df[colunas_anos] > 0).sum(axis=1)

# Criar a coluna Monetário (M) - Soma total dos estoques ao longo do período
df["Monetario"] = df[colunas_anos].sum(axis=1)

# Exibir os dados processados para verificar se está condizente
df.head()


,Sigla,Codigo,Município,1993,1994,1995,1996,1997,1998,1999,...,2019,2020,2021,2022,2023,Data_Inicial,Data_Final,Recencia,Frequencia,Monetario
0,AC,1200013,Acrelândia,0,0,21749,5083039,1180168,3603147,3962375,...,15877934,9088736,8996591,6580221,4908745,1993,2023,0,29,231741082
1,AC,1200054,Assis Brasil,125701,1764378,294535,0,0,244387,688827,...,924183,297710,524263,112330,151097,1993,2023,0,29,41216925
2,AC,1200104,Brasiléia,6372470,5486466,3067611,486008,818832,1006212,2966766,...,10363271,7850346,3423072,3098155,3913477,1993,2023,0,31,177560391
3,AC,1200138,Bujari,0,0,569863,13844,108046,293693,635708,...,14845991,6806347,9281610,4009833,3910707,1993,2023,0,29,257391191
4,AC,1200179,Capixaba,0,0,0,0,104771,560449,0,...,6119740,3826153,2452279,6232797,3550908,1993,2023,0,26,98085601


In [23]:
# Criar uma cópia do dataframe para os cálculos RFM
rfm = df.copy()

# Criar scores RFM por quintis (de 1 a 5)
rfm["score_R"] = pd.qcut(-rfm["Recencia"], 5, labels=False, duplicates="drop") + 1
rfm["score_F"] = pd.qcut(rfm["Frequencia"], 5, labels=False, duplicates="drop") + 1
rfm["score_M"] = pd.qcut(rfm["Monetario"], 5, labels=False, duplicates="drop") + 1

# Criar um score médio entre Frequência e Monetário
rfm["score_FM"] = (rfm["score_F"] + rfm["score_M"]) / 2

# Exibir os dados segmentados usando RFM
rfm.head()


,Sigla,Codigo,Município,1993,1994,1995,1996,1997,1998,1999,...,2023,Data_Inicial,Data_Final,Recencia,Frequencia,Monetario,score_R,score_F,score_M,score_FM
0,AC,1200013,Acrelândia,0,0,21749,5083039,1180168,3603147,3962375,...,4908745,1993,2023,0,29,231741082,1,2,4,3
1,AC,1200054,Assis Brasil,125701,1764378,294535,0,0,244387,688827,...,151097,1993,2023,0,29,41216925,1,2,2,2
2,AC,1200104,Brasiléia,6372470,5486466,3067611,486008,818832,1006212,2966766,...,3913477,1993,2023,0,31,177560391,1,2,4,3
3,AC,1200138,Bujari,0,0,569863,13844,108046,293693,635708,...,3910707,1993,2023,0,29,257391191,1,2,4,3
4,AC,1200179,Capixaba,0,0,0,0,104771,560449,0,...,3550908,1993,2023,0,26,98085601,1,1,3,2


In [26]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Normalizar os dados antes do PCA
scaler = StandardScaler()
df_scaled_pca = scaler.fit_transform(rfm[["Frequencia", "Monetario", "Recencia"]])

# Aplicar PCA para redução de dimensionalidade
pca = PCA()
pca_result = pca.fit_transform(df_scaled_pca)

# Criar a coluna de ranking RFM com base no primeiro componente principal (PCA)
rfm["ranking_RFM"] = np.argsort(-pca_result[:, 0])  # Ordenação decrescente

# Clusterização com K-Means (criando 5 clusters)
kmeans = KMeans(n_clusters=5, n_init=25, random_state=123)
rfm["cluster"] = kmeans.fit_predict(df_scaled_pca)

# Exibir os dados com ranking e cluster
rfm.head()

,Sigla,Codigo,Município,1993,1994,1995,1996,1997,1998,1999,...,Data_Final,Recencia,Frequencia,Monetario,score_R,score_F,score_M,score_FM,ranking_RFM,cluster
0,AC,1200013,Acrelândia,0,0,21749,5083039,1180168,3603147,3962375,...,2023,0,29,231741082,1,2,4,3,5373,0
1,AC,1200054,Assis Brasil,125701,1764378,294535,0,0,244387,688827,...,2023,0,29,41216925,1,2,2,2,1089,0
2,AC,1200104,Brasiléia,6372470,5486466,3067611,486008,818832,1006212,2966766,...,2023,0,31,177560391,1,2,4,3,3270,0
3,AC,1200138,Bujari,0,0,569863,13844,108046,293693,635708,...,2023,0,29,257391191,1,2,4,3,4879,0
4,AC,1200179,Capixaba,0,0,0,0,104771,560449,0,...,2023,0,26,98085601,1,1,3,2,1680,4


In [32]:
# Definição das condições para a classificação RFM no contexto de crédito rural
condicoes = [
    (rfm["score_R"] == 5) & (rfm["score_F"] >= 4) & (rfm["score_M"] >= 4),
    (rfm["score_R"] >= 4) & (rfm["score_F"] >= 3) & (rfm["score_M"] >= 3),
    (rfm["score_R"] == 5) & (rfm["score_F"] < 4) & (rfm["score_M"] < 4),
    (rfm["score_R"] >= 3) & (rfm["score_R"] < 5) & (rfm["score_F"] >= 3) & (rfm["score_M"] >= 3),
    (rfm["score_R"] >= 3) & (rfm["score_F"] >= 2) & (rfm["score_M"] >= 2),
    (rfm["score_R"] == 3) & ((rfm["score_F"] < 2) | (rfm["score_M"] < 2)),
    (rfm["score_R"] == 2) & (rfm["score_F"] >= 3),
    (rfm["score_R"] == 2) & ((rfm["score_F"] < 2) | (rfm["score_M"] < 2)),
    (rfm["score_R"] < 2) & (rfm["score_F"] >= 2) & (rfm["score_M"] >= 2),
    (rfm["score_R"] < 2) & (rfm["score_F"] < 2) & (rfm["score_M"] < 2)
]

# Correspondência das classificações adaptadas
categorias = [
    "Alta Demanda Regular",      # Municípios que frequentemente acessam crédito e têm altos valores acumulados
    "Demanda Consistente",       # Municípios que acessam crédito regularmente, mas em menor volume
    "Demanda Recente",           # Municípios que começaram a acessar crédito recentemente
    "Uso Contínuo",              # Municípios que mantêm um padrão estável de uso de crédito
    "Potencial de Crescimento",  # Municípios com sinais positivos, mas ainda sem alta recorrência
    "Atenção Necessária",        # Municípios que já utilizaram crédito, mas em baixa quantidade
    "Declínio de Crédito",       # Municípios que já tiveram alta frequência, mas estão reduzindo o uso
    "Baixa Demanda",             # Municípios que acessam crédito esporadicamente
    "Declínio Acelerado",        # Municípios com sinais de abandono do crédito
    "Inativo",                   # Municípios que quase não utilizam crédito
]

# Aplicando a classificação ao dataframe
rfm["classificacao_RFM"] = np.select(condicoes, categorias, default="Outros Municípios")

# Exportar o arquivo para Excel
rfm.to_excel("/content/segmentacao_RFM_credito_rural.xlsx", index=False, engine="openpyxl")

# Exibir os dados segmentados
rfm.head()


,Sigla,Codigo,Município,1993,1994,1995,1996,1997,1998,1999,...,Recencia,Frequencia,Monetario,score_R,score_F,score_M,score_FM,ranking_RFM,cluster,classificacao_RFM
0,AC,1200013,Acrelândia,0,0,21749,5083039,1180168,3603147,3962375,...,0,29,231741082,1,2,4,3,5373,0,Declínio Acelerado
1,AC,1200054,Assis Brasil,125701,1764378,294535,0,0,244387,688827,...,0,29,41216925,1,2,2,2,1089,0,Declínio Acelerado
2,AC,1200104,Brasiléia,6372470,5486466,3067611,486008,818832,1006212,2966766,...,0,31,177560391,1,2,4,3,3270,0,Declínio Acelerado
3,AC,1200138,Bujari,0,0,569863,13844,108046,293693,635708,...,0,29,257391191,1,2,4,3,4879,0,Declínio Acelerado
4,AC,1200179,Capixaba,0,0,0,0,104771,560449,0,...,0,26,98085601,1,1,3,2,1680,4,Outros Municípios
